In [2]:
import pandas as pd
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)

/home/tdutton/mambaforge/envs/dsl-final-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_model(
    model_name: str = "meta-llama/Meta-Llama-3-8B-Instruct", load_on_gpu: bool = True
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    if load_on_gpu:
        config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_name, low_cpu_mem_usage=True, quantization_config=config
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer

In [4]:
# Load the model & tokenizer
model, tokenizer = load_model()

Loading checkpoint shards: 100%|██████████| 4/4 [00:34<00:00,  8.75s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Add Lora Matrices
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

peft_model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./output",
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1,
    report_to="wandb",  ## can be wandb, but we are reporint to noe
)

trainer = Trainer(model=peft_model, 
                      args=training_args,
                      train_dataset=tokenize_datasets['train']
        )